In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import time
from datetime import datetime
import matplotlib.pyplot as plt

 
plt.style.use(['science','notebook'])
plt.style.reload_library()

In [3]:
from june import World 
from june.geography import Geography
from june.demography import Demography
from june.interaction import Interaction
from june.infection import Infection, HealthIndexGenerator, InfectionSelector
from june.infection.transmission import TransmissionConstant
from june.groups import Hospitals, Schools, Companies, Households, CareHomes, Cemeteries, Universities
from june.groups.leisure import generate_leisure_for_config, Cinemas, Pubs, Groceries
from june.groups.travel import *
from june.simulator import Simulator
from june.infection_seed import InfectionSeed
from june.policy import Policy, Policies
from june import paths
from june.hdf5_savers import load_geography_from_hdf5
from june.records import Record, RecordReader

from june.world import generate_world_from_geography
from june.hdf5_savers import generate_world_from_hdf5

No --data argument given - defaulting to:
/home/florpi/JUNE/data
No --configs argument given - defaulting to:
/home/florpi/JUNE/june/configs
INFO:numexpr.utils:NumExpr defaulting to 4 threads.


# Initialize world

To initialize a certain world, we need to add the different components we want to have in it. First we specify what super areas (msoa) we want to create. We have included these ones, because they are known to contain hospitals, schools, care homes, and companies.

After creating the geography, we create the different components the worlds need to have such as care homes, companies ...

In [4]:
CONFIG_PATH = paths.configs_path / "config_example.yaml"

In [8]:
%%time 

geography = Geography.from_file(
{
    "super_area": ["E02001731", "E02002566","E02004935","E02000134", "E02004987"]
}
)

geography.hospitals = Hospitals.for_geography(geography)
geography.schools = Schools.for_geography(geography)
geography.companies = Companies.for_geography(geography)
geography.care_homes = CareHomes.for_geography(geography)
#geography.universities = Universities.for_super_areas(geography.super_areas)
world = generate_world_from_geography(geography, include_households=True)


2020-11-01 16:46:02,708 - june.geography.geography - INFO - There are 151 areas and 5 super_areas and 3 in the world.
2020-11-01 16:46:02,714 - hospitals - INFO - There are 1 hospitals in this geography.
2020-11-01 16:46:02,765 - schools - INFO - There are 16 schools in this geography.
2020-11-01 16:46:02,815 - schools - INFO - No school for the age 0 in this world.
2020-11-01 16:46:02,825 - schools - INFO - No school for the age 1 in this world.
2020-11-01 16:46:03,120 - care_homes - INFO - There are 14 care_homes in this geography.
2020-11-01 16:46:09,186 - world - INFO - Populating areas
2020-11-01 16:46:09,764 - world - INFO - Areas populated. This world's population is: 44314
2020-11-01 16:46:12,116 - worker_distributor - INFO - Distributing workers to super areas...
2020-11-01 16:46:13,656 - worker_distributor - INFO - Workers distributed.
2020-11-01 16:46:14,517 - care_home_distributor - INFO - Populating care homes
2020-11-01 16:46:14,524 - care_home_distributor - INFO - This w

## Commute, travel and leisure

In [9]:
%%time

world.pubs = Pubs.for_geography(geography)
world.cinemas = Cinemas.for_geography(geography)
world.groceries = Groceries.for_geography(geography)
leisure = generate_leisure_for_config(world, config_filename=CONFIG_PATH)
leisure.distribute_social_venues_to_areas(
    areas=world.areas, super_areas=world.super_areas
)


2020-11-01 16:46:24,849 - social_venue - INFO - Domain 0 has 12888 pubs(s)
2020-11-01 16:46:24,920 - social_venue - INFO - Domain 0 has 60 cinemas(s)
2020-11-01 16:46:26,392 - social_venue - INFO - Domain 0 has 3136 groceries(s)
2020-11-01 16:46:26,451 - leisure - INFO - Linking households for visits
2020-11-01 16:46:27,262 - leisure - INFO - Done
2020-11-01 16:46:27,263 - leisure - INFO - Linking households with care homes for visits
2020-11-01 16:46:27,387 - leisure - INFO - Done
2020-11-01 16:46:27,388 - leisure - INFO - Distributing social venues to areas
2020-11-01 16:46:27,389 - leisure - INFO - Distributed in 0 of 151 areas.
2020-11-01 16:46:27,830 - leisure - INFO - Distributed in 151 of 151 areas.
CPU times: user 7.73 s, sys: 68.6 ms, total: 7.79 s
Wall time: 7.86 s


In [10]:
# initialise commuting travel
travel = Travel()
travel.initialise_commute(world)

2020-11-01 16:46:27,871 - travel - INFO - Initialising commute...
2020-11-01 16:46:27,872 - travel - INFO - Creating cities...
2020-11-01 16:46:27,885 - travel - INFO - This world has 4 cities, with names
['London', 'Newcastle upon Tyne', 'Darlington', 'St Albans']
2020-11-01 16:46:27,890 - travel - INFO - Determining people mode of transport
2020-11-01 16:46:36,903 - travel - INFO - Mode of transport allocated in 0 of 151 areas.
2020-11-01 16:46:36,991 - travel - INFO - Mode of transport determined for everyone.
2020-11-01 16:46:37,249 - travel - INFO - Assigning commuters to stations...
2020-11-01 16:46:37,250 - travel - INFO - Assigned 0 of 44314 potential commuters...
2020-11-01 16:46:37,273 - travel - INFO - Commuters assigned
2020-11-01 16:46:37,274 - travel - INFO - City London has 742 internal and 797 external commuters.
2020-11-01 16:46:37,275 - travel - INFO - City Newcastle upon Tyne has 1085 internal and 1031 external commuters.
2020-11-01 16:46:37,279 - travel - INFO - Cre

We are also going to need some cemeteries...


In [11]:
world.cemeteries = Cemeteries()

In [12]:
len(world.people)

44314

### If it took a long time to run the previous commands, it might be a good idea to save the world to reuse it later.

In [13]:
world.to_hdf5("world.hdf5")

2020-11-01 16:46:37,434 - world_saver - INFO - saving world to HDF5
2020-11-01 16:46:37,459 - world_saver - INFO - saving population...
2020-11-01 16:46:38,898 - world_saver - INFO - saving hospitals...
2020-11-01 16:46:38,903 - world_saver - INFO - saving schools...
2020-11-01 16:46:38,912 - world_saver - INFO - saving companies...
2020-11-01 16:46:38,945 - world_saver - INFO - saving households...
2020-11-01 16:46:39,302 - world_saver - INFO - saving care homes...
2020-11-01 16:46:39,306 - world_saver - INFO - saving cities...
2020-11-01 16:46:39,313 - world_saver - INFO - saving stations...
2020-11-01 16:46:39,319 - world_saver - INFO - saving social venues...


If we would like to load the world we saved, we just do

In [14]:
world = generate_world_from_hdf5("world.hdf5")

2020-11-01 16:46:39,435 - world_saver - INFO - loading world from HDF5
2020-11-01 16:46:39,459 - world_saver - INFO - loading hospitals...
2020-11-01 16:46:39,468 - world_saver - INFO - loading schools...
2020-11-01 16:46:39,480 - company_saver - INFO - loading companies...
2020-11-01 16:46:39,482 - company_saver - INFO - Companies chunk 0 of 1
2020-11-01 16:46:39,595 - world_saver - INFO - loading care homes...
2020-11-01 16:46:39,600 - world_saver - INFO - loading cities...
2020-11-01 16:46:39,611 - world_saver - INFO - loading stations...
2020-11-01 16:46:39,630 - household_saver - INFO - loading households...
2020-11-01 16:46:39,634 - household_saver - INFO - Loaded chunk 0 of 1
2020-11-01 16:46:39,908 - population saver - INFO - loading population...
2020-11-01 16:46:39,909 - population saver - INFO - Loaded chunk 0 of 1
2020-11-01 16:46:40,328 - world_saver - INFO - loading social venues...
2020-11-01 16:46:40,334 - social_venue - INFO - Domain 0 has 60 cinemas(s)
2020-11-01 16:4

In [15]:
# and regenerate leisure in case we load it externally
leisure = generate_leisure_for_config(world, CONFIG_PATH)
# create travel as well
travel = Travel()

you have now a beautiful pre-pandemic world. 

# Adding the infection

The module in charge of infecting people is called the ``InfectionSelector``, which gives people a transmission time profile and a symptoms trajectory based on their age and sex (through the health index generator)

In [16]:
health_index_generator = HealthIndexGenerator.from_file(asymptomatic_ratio=0.2)
selector = InfectionSelector.from_file(
        health_index_generator=health_index_generator,
        transmission_config_path=paths.configs_path / 'defaults/transmission/XNExp.yaml'
)

# Adding the interaction

In [17]:
interaction = Interaction.from_file(population=world.people)

Beta are the intensities of the interaction taking place at the different groups

In [18]:
interaction.beta

{'box': 1,
 'pub': 0.42941,
 'grocery': 0.04137,
 'cinema': 0.157461,
 'city_transport': 0.107969,
 'inter_city_transport': 0.383,
 'hospital': 0.1168,
 'care_home': 0.28,
 'company': 0.371,
 'school': 0.07,
 'household': 0.208,
 'university': 0.306}

moreover this interaction module uses contact matrices, that are different for different groups. These contact matrices shouldnt be modified for now. However they are a combination of conversational contact matrices, and physical contact matrices (see the BBC pandemic paper, from where these matrices are extracted https://www.medrxiv.org/content/10.1101/2020.02.16.20023754v2)

There is a parameter, ``alpha`` ($\alpha$), that combines these two matrices in the following way,


$\beta M \left(1 + (\alpha -1) \right) P$

where $\beta$ is the intensity of the interaction, and $P$ the physical contact matrix. A larger $\alpha$ produces more physical contacts. It is an overall number, non dependent of the particular group.


In [19]:
interaction.alpha_physical

2.0

# Seed the disease

There are two options implemented in the seed at the moment, either you specify the number of cases and these are then homogeneously distributed by population to the different areas, or you use UK data on cases per region. For now use the first case.

In [20]:
infection_seed = InfectionSeed(
    world, selector,
)

In [21]:
n_cases = 50
infection_seed.unleash_virus(
    population=world.people,
    n_cases=n_cases) # play around with the initial number of cases

# Set policies

In [22]:
policies = Policies.from_file()

We can have a look at one of the policies

In [23]:
print(policies.individual_policies[1].__dict__)

{'spec': 'limit_long_commute', 'start_time': datetime.datetime(1000, 1, 1, 0, 0), 'end_time': datetime.datetime(9999, 1, 1, 0, 0), 'policy_type': 'individual', 'policy_subtype': 'skip_activity', 'activities_to_remove': ['primary_activity', 'commute'], 'going_to_work_probability': 0.2}


# Run the simulation

The simulator is the main module in charge of running the simulation. It coordinates the ``ActivityManager`` which is responsible of allocating people to the right groups given the current timestep, it updates the health status of the population, and it runs the interaction over the different groups. All of these modules can be modified by policies at any given time.

Since the timer configuration is a bit cumbersome, it is read from the config file at ``configs/config_example.yaml``

In [24]:
record = Record(    
    record_path = 'results',    
    record_static_data=True,
) 

In [25]:
record.static_data(world=world)

In [30]:
simulator = Simulator.from_file(
    world=world,
    infection_selector=selector,
    interaction=interaction, 
    config_filename = CONFIG_PATH,
    leisure = leisure,
    travel = travel,
    record=record,
    policies = policies
)

In [ ]:
%%time
simulator.run()

2020-11-01 16:48:20,253 - simulator - INFO - Starting simulation for 30 days at day 2020-03-01 00:00:00, to run for 30 days
2020-11-01 16:48:20,500 - simulator - INFO - ==================== timestep ====================
2020-11-01 16:48:21,046 - activity_manager - INFO - CMS: People COMS for rank 0/1 - 4.837987944483757e-06,4.0531158447265625e-06 - 2020-03-01 00:00:00
2020-11-01 16:48:21,052 - simulator - INFO - Info for rank 0, Date = 2020-03-01 00:00:00, number of deaths =  0, number of infected = 488
2020-11-01 16:48:21,306 - simulator - INFO - CMS: Infection COMS-v2 for rank 0/1(2) 0.0001253569935215637,0.00012493133544921875 - 2020-03-01 00:00:00
2020-11-01 16:48:21,421 - simulator - INFO - CMS: Timestep for rank 0/1 - 0.9192704309971305, 0.9192705154418945 - 2020-03-01 00:00:00

2020-11-01 16:48:21,422 - simulator - INFO - ==================== timestep ====================
2020-11-01 16:48:21,829 - activity_manager - INFO - CMS: People COMS for rank 0/1 - 4.538000212050974e-06,4.

2020-11-01 16:48:31,209 - activity_manager - INFO - CMS: People COMS for rank 0/1 - 4.150002496317029e-06,3.814697265625e-06 - 2020-03-03 10:00:00
2020-11-01 16:48:31,215 - simulator - INFO - Info for rank 0, Date = 2020-03-03 10:00:00, number of deaths =  0, number of infected = 584
2020-11-01 16:48:31,491 - simulator - INFO - CMS: Infection COMS-v2 for rank 0/1(2) 0.00025388300127815455,0.0002536773681640625 - 2020-03-03 10:00:00
2020-11-01 16:48:31,638 - simulator - INFO - CMS: Timestep for rank 0/1 - 1.1497294830041938, 1.1497325897216797 - 2020-03-03 10:00:00

2020-11-01 16:48:31,639 - simulator - INFO - ==================== timestep ====================
2020-11-01 16:48:31,891 - activity_manager - INFO - CMS: People COMS for rank 0/1 - 8.107002940960228e-06,7.62939453125e-06 - 2020-03-03 13:00:00
2020-11-01 16:48:31,900 - simulator - INFO - Info for rank 0, Date = 2020-03-03 13:00:00, number of deaths =  0, number of infected = 590
2020-11-01 16:48:32,131 - simulator - INFO - CMS

2020-11-01 16:48:40,916 - simulator - INFO - CMS: Infection COMS-v2 for rank 0/1(2) 0.0001357160072075203,0.00013494491577148438 - 2020-03-06 00:00:00
2020-11-01 16:48:41,051 - simulator - INFO - CMS: Timestep for rank 0/1 - 0.6172755859879544, 0.6172771453857422 - 2020-03-06 00:00:00

2020-11-01 16:48:41,053 - simulator - INFO - ==================== timestep ====================
2020-11-01 16:48:41,367 - activity_manager - INFO - CMS: People COMS for rank 0/1 - 5.3440016927197576e-06,4.5299530029296875e-06 - 2020-03-06 01:00:00
2020-11-01 16:48:41,374 - simulator - INFO - Info for rank 0, Date = 2020-03-06 01:00:00, number of deaths =  0, number of infected = 1019
2020-11-01 16:48:41,739 - simulator - INFO - CMS: Infection COMS-v2 for rank 0/1(2) 0.0003870069922413677,0.00039076805114746094 - 2020-03-06 01:00:00
2020-11-01 16:48:41,898 - simulator - INFO - CMS: Timestep for rank 0/1 - 0.8417852989950916, 0.8417859077453613 - 2020-03-06 01:00:00

2020-11-01 16:48:41,899 - simulator - I

2020-11-01 16:48:52,042 - simulator - INFO - CMS: Timestep for rank 0/1 - 0.6618557080073515, 0.6618564128875732 - 2020-03-08 12:00:00

2020-11-01 16:48:52,042 - simulator - INFO - ==================== timestep ====================
2020-11-01 16:48:52,305 - activity_manager - INFO - CMS: People COMS for rank 0/1 - 6.540009053424001e-06,5.7220458984375e-06 - 2020-03-09 00:00:00
2020-11-01 16:48:52,317 - simulator - INFO - Info for rank 0, Date = 2020-03-09 00:00:00, number of deaths =  0, number of infected = 1887
2020-11-01 16:48:52,565 - simulator - INFO - CMS: Infection COMS-v2 for rank 0/1(2) 0.00017142300202976912,0.00017070770263671875 - 2020-03-09 00:00:00
2020-11-01 16:48:52,744 - simulator - INFO - CMS: Timestep for rank 0/1 - 0.701261402995442, 0.7012619972229004 - 2020-03-09 00:00:00

2020-11-01 16:48:52,745 - simulator - INFO - ==================== timestep ====================
2020-11-01 16:48:53,103 - activity_manager - INFO - CMS: People COMS for rank 0/1 - 6.517002475447

2020-11-01 16:49:03,812 - simulator - INFO - ==================== timestep ====================
2020-11-01 16:49:04,089 - activity_manager - INFO - CMS: People COMS for rank 0/1 - 6.583999493159354e-06,5.7220458984375e-06 - 2020-03-11 09:00:00
2020-11-01 16:49:04,095 - simulator - INFO - Info for rank 0, Date = 2020-03-11 09:00:00, number of deaths =  2, number of infected = 4577
2020-11-01 16:49:04,634 - simulator - INFO - CMS: Infection COMS-v2 for rank 0/1(2) 0.0001393529964843765,0.00013899803161621094 - 2020-03-11 09:00:00
2020-11-01 16:49:04,774 - simulator - INFO - CMS: Timestep for rank 0/1 - 0.9616012869955739, 0.961601972579956 - 2020-03-11 09:00:00

2020-11-01 16:49:04,775 - simulator - INFO - ==================== timestep ====================
2020-11-01 16:49:05,371 - activity_manager - INFO - CMS: People COMS for rank 0/1 - 4.956993507221341e-06,4.0531158447265625e-06 - 2020-03-11 10:00:00
2020-11-01 16:49:05,377 - simulator - INFO - Info for rank 0, Date = 2020-03-11 10:0

2020-11-01 16:49:13,384 - simulator - INFO - Info for rank 0, Date = 2020-03-13 13:00:00, number of deaths =  4, number of infected = 8488
2020-11-01 16:49:13,672 - simulator - INFO - CMS: Infection COMS-v2 for rank 0/1(2) 0.00016922900977078825,0.0001685619354248047 - 2020-03-13 13:00:00
2020-11-01 16:49:13,836 - simulator - INFO - CMS: Timestep for rank 0/1 - 0.6189798550039995, 0.6189804077148438 - 2020-03-13 13:00:00

2020-11-01 16:49:13,837 - simulator - INFO - ==================== timestep ====================
2020-11-01 16:49:14,254 - activity_manager - INFO - CMS: People COMS for rank 0/1 - 3.883993485942483e-06,3.337860107421875e-06 - 2020-03-14 00:00:00
2020-11-01 16:49:14,262 - simulator - INFO - Info for rank 0, Date = 2020-03-14 00:00:00, number of deaths =  6, number of infected = 8886
2020-11-01 16:49:14,592 - simulator - INFO - CMS: Infection COMS-v2 for rank 0/1(2) 0.00012412300566211343,0.00012350082397460938 - 2020-03-14 00:00:00
2020-11-01 16:49:14,735 - simulator -

While the simulation runs (and afterwards) we can launch the visualization webpage by running
```python june/visualizer.py path/to/results``` 

# Getting the results

All results are stored in a json file specified in the ``record.record_path`` folder. Summaries are found under ``summary.csv``

In [ ]:
import pandas as pd

In [ ]:
read = RecordReader()

## Contains summaries with regional information

In [ ]:
read.regional_summary.head(3)

In [ ]:
read.regional_summary['daily_intensive_care'].sum()

In [ ]:
for region in read.regional_summary['region'].unique():
    read.regional_summary[
        read.regional_summary['region'] == region
    ]['current_infected'].plot()
    read.regional_summary[
        read.regional_summary['region'] == region
    ]['current_susceptible'].plot()
    plt.title(region)
    plt.show()

In [ ]:
read.world_summary['current_infected'].plot()
read.world_summary['current_susceptible'].plot()

# Asking questions to the records

## Sero-prevalence by age

In [ ]:
infections_df = read.get_table_with_extras('infections',
                                           'infected_ids')

In [ ]:
deaths_df = read.get_table_with_extras('deaths', 
                                       'dead_person_ids')

In [ ]:
age_bins = (0,20,60,100)

In [ ]:
infected_by_age = infections_df.groupby([pd.cut(infections_df['age'],
            bins=age_bins), 'timestamp']).size()

In [ ]:
people_df = read.table_to_df('population')

In [ ]:
n_by_age = people_df.groupby(pd.cut(people_df['age'],
            bins=age_bins)).size()

In [ ]:
(100*infected_by_age/n_by_age).xs(10).cumsum().plot(label='0,20')
(100*infected_by_age/n_by_age).xs(30).cumsum().plot(label='20,60')
(100*infected_by_age/n_by_age).xs(70).cumsum().plot(label='60,100')
plt.legend()

## Care home deaths in hospital

In [ ]:
care_home_deaths_hospital = deaths_df[
    (deaths_df['location_specs'] == 'hospital') 
    & (deaths_df['residence_type'] == 'care_home')
]
care_home_deaths_hospital=care_home_deaths_hospital.groupby(
    ['name_region', 'timestamp']
).size()

In [ ]:
care_home_deaths_hospital.unstack(level=0).plot()


## Where people get infected as a function of time

In [ ]:
locations_df = infections_df.groupby(['location_specs', 
                                'timestamp']).size()

In [ ]:
locations_df.unstack(level=0).plot()

In [ ]:
import matplotlib.ticker as mtick
location_counts_df = locations_df.groupby('location_specs').size()
location_counts_df = 100*location_counts_df / location_counts_df.sum()
ax = location_counts_df.sort_values().plot.bar()
ax.yaxis.set_major_formatter(mtick.PercentFormatter())
plt.ylabel('Percentage of infections at location')
plt.xlabel('location')


## Where people of certain age get infected as a function of time

In [ ]:
old_locations_df = infections_df[
    infections_df.age > 65
].groupby(['location_specs', 'timestamp']).size()

In [ ]:
old_locations_df.unstack(level=0).plot()

## Prevalence by household size

In [ ]:
household_people = people_df[
    people_df['residence_type'] == 'household'
]

In [ ]:
household_sizes = household_people.groupby('residence_id').size()

In [ ]:
household_sizes.hist() # in units of households

In [ ]:
household_people.loc[:,'household_size'] = household_sizes.loc[
    household_people['residence_id']
].copy(deep=True).values

In [ ]:
household_people['household_size'].hist() # in units of people

In [ ]:
household_infections_df = infections_df.merge(
    household_people['household_size'], 
    left_index=True, right_index=True, how='inner'
)

In [ ]:
(household_infections_df.groupby(
    'household_size'
).size()/household_people.groupby('household_size').size()).plot()
plt.xlabel('Household size')
plt.ylabel('% of people infected by household size')
plt.xlim(0,8)

In [ ]:
# How many households have everyone infected?

In [ ]:
n_infected_by_household = infections_df[
    infections_df['residence_type'] == 'household'
].groupby('residence_id').size()

In [ ]:
n_total_in_household = household_people[
    household_people['residence_id'].isin(
        n_infected_by_household.index
    )
].groupby('residence_id').size()

In [ ]:
(n_infected_by_household/n_total_in_household).hist()
plt.xlabel('% of the household infected')

## Percentage of infected per care home

In [ ]:
n_infected_by_carehome = infections_df[
    infections_df['residence_type'] == 'care_home'
].groupby(
    'residence_id'
).size()

In [ ]:
n_total_in_carehome = people_df[
    (people_df['residence_type'] == 'care_home') 
    & (people_df['residence_id'].isin(n_infected_by_carehome.index))  
].groupby('residence_id').size()

In [ ]:
(n_infected_by_carehome/n_total_in_carehome).hist()
plt.xlabel('% of the care home infected')

In [ ]:
# from all care homes, how many got at least one case?

In [ ]:
n_total_care_homes = people_df[
    (people_df['residence_type'] == 'care_home') 
]['residence_id'].nunique()

In [ ]:
n_total_care_homes

In [ ]:
care_homes_with_infected = infections_df[
    (infections_df['residence_type'] == 'care_home') 
]['residence_id'].nunique()

In [ ]:
care_homes_with_infected/n_total_care_homes